In [1]:
import pandas as pd
import yfinance as yf
from pathlib import Path
import concurrent.futures

In [2]:
# Pulling S&P Data from wiki and outputing HTML
# Sepecify URL
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Read html
sp500_html = pd.read_html(url)

# Obtain first table
sp500_html = sp500_html[0]

# Create dataframe
sp500_df = pd.DataFrame(sp500_html)
sp500_df.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [3]:
sp500_all_sectors_df = pd.DataFrame(
    columns=['GICS Sector', 'Symbol'],
    data=sp500_df,
    )

sp500_df_wo_index = sp500_all_sectors_df.set_index("Symbol")

# isolate symbols in order to pass list to yfinance to get market cap info
sp500_all_symbols = sp500_all_sectors_df['Symbol'].values.tolist()

# Wikipedia symbols with a "." instead of a "-"
# Yfinance needs the "-" to pull data
stocks = []

for stock_ticker in sp500_all_symbols:
    ticker = stock_ticker.replace(".","-")
    stocks.append(ticker)


In [ ]:
market_cap_dict = {}
exception_count = 0
cap_count = 0
exception_list = []
key_error_list = []

def get_stock_market_cap(stock):
#     print(f'\rGetting {stock} ticker...', end='')
    ticker = yf.Ticker(stock)
    return ticker.info['marketCap']
      
# with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_mkt_cap = {executor.submit(get_stock_market_cap, stock): stock for stock in stocks}
    for future in concurrent.futures.as_completed(future_to_mkt_cap):
        stock = future_to_mkt_cap[future]
#         print(stock, future.result())
        try:
            market_cap_dict[stock] = future.result()
            cap_count += 1
        except KeyError:
            key_error_list.append(stock)
        except Exception as exc:
            exception_count += 1
            exception_list.append(stock)
            print(f'\n{stock} generated a {type(exc)} exception: {exc}', end='\n')
        else:
            print(f'\r{stock}: {market_cap_dict[stock]}', end='')
    print(f'\rDone! {cap_count} stocks, {len(key_error_list)} key errors, {exception_count} unhandled exceptions.')
    print(f'Tickers with no market cap data:\n{key_error_list}')


In [5]:
# Return a sorted Pandas DataFrame based on market cap
# The columns will originally be the ticker, use ".T" to transpose the table
# Use .sort_values to sort by column [0] in decending order
market_cap_df = pd.DataFrame(
    market_cap_dict, 
    index=[0],
).T.sort_values(
    by=[0], 
    ascending=False,
)

# Rename the column and index to be merged
market_cap_df.columns = ['Market_Cap']
market_cap_df.index.names = ['Symbol']
display(market_cap_df.head())

,Market_Cap
Symbol,
AAPL,2433886453760
GOOGL,1898919362560
GOOG,1853121495040
AMZN,1704640380928
BRK-B,628953186304


In [6]:
# merge sp500_df_wo_index and market_cap_df to create 1 complete data frame 
# to be sliced for analysis
stock_industry_marketcap = pd.merge(
    sp500_df_wo_index, 
    market_cap_df, 
    left_index=True, 
    right_index=True
)

stock_industry_marketcap.head()

,GICS Sector,Market_Cap
Symbol,,
MMM,Industrials,104756625408
ABT,Health Care,220485632000
ABBV,Health Care,187943485440
ABMD,Health Care,15899406336
ACN,Information Technology,215118315520


In [7]:
stock_industry_marketcap.sort_values(by=['GICS Sector', 'Market_Cap'], ascending=False)

,GICS Sector,Market_Cap
Symbol,,
AEP,Utilities,41339322368
AWK,Utilities,32086677504
AEE,Utilities,21356564480
AES,Utilities,15619478528
LNT,Utilities,14367754240
...,...,...
AAP,Consumer Discretionary,13586731008
GOOGL,Communication Services,1898919362560
GOOG,Communication Services,1853121495040


In [8]:
stock_industry_marketcap.head()

,GICS Sector,Market_Cap
Symbol,,
MMM,Industrials,104756625408
ABT,Health Care,220485632000
ABBV,Health Care,187943485440
ABMD,Health Care,15899406336
ACN,Information Technology,215118315520


In [9]:
# save new dataframe to csv to be used in other code
stock_industry_marketcap.to_csv("stock_industry_marketcap.csv")